---
title: "CAPTAIN visualisation workflow"
author: "Théophile L. Mouton"
date: "September 24, 2024"
format:
  html:
    toc: true
    toc-location: right
    css: custom.css
    output-file: "CAPTAIN_workflow.html"
    self-contained: true
    code-fold: true
    code-tools: true
editor: visual
execute:
  warning: false
  message: false
  echo: true
jupyter: myenv
---


# Visualise CAPTAIN results 

## Open R libraries


```{r}
library(readr)
library(ggplot2)
library(sf)
library(rnaturalearth)
library(rnaturalearthdata)
library(dplyr)
library(gridExtra)
library(biscale)
library(colorspace)
library(grid)
```


## CAPTAIN on EDGE 


```{r}
# Read the RDS file from the Data folder
data <- readRDS("Data/EDGE_full_results_averaged_budget0.01_replicates11.rds")

# Get world map data
world <- ne_countries(scale = "medium", returnclass = "sf")

# Define the McBryde-Thomas 2 projection
mcbryde_thomas_2 <- "+proj=mbt_s"

# Transform the data to sf object and project
data_sf <- st_as_sf(data, coords = c("Longitude", "Latitude"), crs = 4326) %>%
  st_transform(crs = mcbryde_thomas_2)

# Project the world map
world_projected <- st_transform(world, crs = mcbryde_thomas_2)

# Create the outline from the projected world map
outline <- st_union(world_projected) %>% st_boundary()

# Create the globe bounding box
globe_bbox <- rbind(c(-180, -90), c(-180, 90), 
                    c(180, 90), c(180, -90), c(-180, -90))

# Create the globe border
globe_border <- st_polygon(list(globe_bbox)) %>%
  st_sfc(crs = 4326) %>%
  st_sf(data.frame(rgn = 'globe', geom = .)) %>%
  smoothr::densify(max_distance = 0.5) %>%
  st_transform(crs = mcbryde_thomas_2)

# Create the plot
ggplot() +
  # Add points for each planning unit, colored by priority
  geom_sf(data = data_sf, aes(color = Priority), size = 0.5, alpha = 0.7) +
  # Add the world map with a light gray fill
  geom_sf(data = world_projected, fill = "lightgrey", color = "lightgrey", size = 0.1) +
  # Add the black outline around the globe
  geom_sf(data = globe_border, fill = NA, color = "black", size = 0.5) +  # Add the globe border
  # Use a white to yellow to blue color gradient
  scale_color_gradientn(
    colors = c("white", "yellow", "darkblue"),
    values = c(0, 0.5, 1),
    name = "Priority",
    guide = guide_colorbar(barwidth = 20, barheight = 0.5, 
                           title.position = "top", title.hjust = 0.5)
  ) +
  # Add labels and title
  labs(title = "Global Distribution of Conservation Priorities",
       subtitle = "Index: EDGE2, Budget: 0.01, Replicates: 11",
       x = "Longitude", y = "Latitude") +
  # Adjust theme
  theme_minimal() +
  theme(
    legend.position = "bottom",
    legend.direction = "horizontal",
    legend.box = "vertical",
    legend.margin = margin(t = 20, r = 0, b = 0, l = 0),
    legend.title = element_text(margin = margin(b = 10)),
    panel.background = element_rect(fill = "white", color = NA),
    plot.background = element_rect(fill = "white", color = NA),
    panel.grid = element_blank()
  )
```


## Run CAPTAIN on EDGE2


```{r}
```


## Run CAPTAIN on FUSE


```{r}
```